In [ ]:
import os
import shutil
import tempfile
import argparse
from concurrent import futures

from ccdc.docking import Docker
from ccdc.io import MoleculeReader

In [ ]:
protein='/media/boltjes/drugdesign3/angel/ZND/Data/5lgd.mol2'
ligands_folder='/media/boltjes/drugdesign3/angel/ZND/Ligands/'
out_folder='/media/boltjes/drugdesign3/angel/ZND/Dockings/'

In [ ]:
def run_gold (params):
    (protein,ligand,out_folder) = params
    try:
        print ('Processing:',ligand.split('.')[0])
        docker = Docker()
        settings = docker.settings
        settings.add_protein_file (protein)
        settings.add_ligand_file (ligands_folder+ligand)
        settings.binding_site = settings.BindingSiteFromPoint (protein=protein, origin=(-46.1,-34.37,22), distance=10.0)
        settings.fitness_function = 'plp'
        settings.rescore_function = 'goldscore'
        settings.autoscale = 10.0
        settings.early_termination = False
        tmp_folder = tempfile.mkdtemp(prefix=ligand.split('.')[0])
        settings.output_directory = tmp_folder
        settings.output_file = (ligand)
        results = docker.dock()
        shutil.move (settings.conf_file, out_folder+ligand.split('.')[0] +'.conf')
        shutil.move (tmp_folder+'/'+ligand, out_folder)
        shutil.rmtree(tmp_folder)
    
    except Exception:
        print (ligand.split('.')[0],'Could not been processed')
            
            
with futures.ProcessPoolExecutor(max_workers=2) as executor:
    ligands=[]
    docked= []

    for out in os.listdir(out_folder):
        if '.mol2' in out:
            docked.append (out.split('.')[0])

    for ligand in sorted (os.listdir (ligands_folder)):
        if not (ligand.split('.')[0]) in docked:
            if '.mol2' in ligand:
                ligands.append (ligand)

    for ligand in ligands:
        job =executor.submit (run_gold, (protein,ligand,out_folder),)
        
    print(job.result())